In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import yaml
from IPython.display import Image

from config import FIG_RESULTS_DIR
from drawutil import utcolors, mpl_config
from model import Solution
from util import obj_breakdown

In [ ]:
# global setting
SCALE = 1 # scale = 1.0 for write-up dimensions
mpl_config(SCALE)

In [ ]:
casestudies = [
    'imelda',
    'harvey',
]

pftypes = {
    'imelda': [
        'dc',
        'lpacc',
        'lpacf',
        'qpac',
    ],
    'harvey': [
        'dc',
        'lpacc',
    ]
}

pftype_names = {
    'dc': 'DC',
    'lpacc': 'LPAC-C',
    'lpacf': 'LPAC-F',
    'qpac': 'QPAC',
}

approaches = [
    'stochastic',
    'robust',
]

f_mins = {
    ('stochastic', 'imelda'): 0,
    ('robust', 'imelda'): 0,
    ('stochastic', 'harvey'): 0,
    ('robust', 'harvey'): 0,
}

f_maxs = {
    ('stochastic', 'imelda'): 20,
    ('robust', 'imelda'): 20,
    ('stochastic', 'harvey'): 193,
    ('robust', 'harvey'): 193,
}

r_hat = 3

xticks = {
    ('stochastic', 'imelda'): np.arange(0, 20 + 5, 5),
    ('robust', 'imelda'): np.arange(0, 20 + 5, 5),
    ('stochastic', 'harvey'): np.arange(0, 193 + 50, 50),
    ('robust', 'harvey'): np.arange(0, 193 + 50, 50),
}

yticks = {
    ('stochastic', 'imelda'): np.arange(0.6, 0.9 + 0.1, 0.1),
    ('robust', 'imelda'): np.arange(0.6, 1.2 + 0.2, 0.2),
    ('stochastic', 'harvey'): np.arange(1.4, 3.2 + 0.6, 0.6),
    ('robust', 'harvey'): np.arange(3.7, 5.5 + 0.6, 0.6),
}

xlim = {
    ('stochastic', 'imelda'): (0 - 20 * 0.05, 20 + 20 * 0.05),
    ('robust', 'imelda'): (0 - 20 * 0.05, 20 + 20 * 0.05),
    ('stochastic', 'harvey'): (0 - 200 * 0.05, 200 + 200 * 0.05),
    ('robust', 'harvey'): (0 - 200 * 0.05, 200 + 200 * 0.05),
}

ylim = {
    ('stochastic', 'imelda'): (0.6 - 0.3 * 0.05, 0.9 + 0.3 * 0.05),
    ('robust', 'imelda'): (0.6 - 0.6 * 0.05, 1.2 + 0.6 * 0.05),
    ('stochastic', 'harvey'): (1.4 - 1.8 * 0.05, 3.2 + 1.8 * 0.05),
    ('robust', 'harvey'): (3.7 - 1.8 * 0.05, 5.5 + 1.8 * 0.05),
}

In [ ]:
for casestudy in casestudies:

    # subplot sizing overhead
    plot_num_w = 2
    plot_num_h = len(pftypes[casestudy])
    lo_pad_w, md_pad_w, hi_pad_w, plot_dim_w = 0.750 * SCALE, 1.000 * SCALE, 0.250 * SCALE, 2.500 * SCALE
    lo_pad_h, md_pad_h, hi_pad_h, plot_dim_h = 0.500 * SCALE, 0.250 * SCALE, 0.250 * SCALE, 1.500 * SCALE
    fig_w = lo_pad_w + hi_pad_w + md_pad_w * (plot_num_w - 1) + plot_dim_w * plot_num_w
    fig_h = lo_pad_h + hi_pad_h + md_pad_h * (plot_num_h - 1) + plot_dim_h * plot_num_h
    fig, axes = plt.subplots(plot_num_h, plot_num_w,
                             figsize=(fig_w, fig_h),
                             sharex='col',
                             sharey=False)
    plt.subplots_adjust(left=lo_pad_w / fig_w,
                        bottom=lo_pad_h / fig_h,
                        right=(fig_w - hi_pad_w) / fig_w,
                        top=(fig_h - hi_pad_h) / fig_h,
                        wspace=md_pad_w / plot_dim_w,
                        hspace=md_pad_h / plot_dim_h)
    
    for row, pftype in enumerate(pftypes[casestudy]):

        f_min = f_mins['stochastic', casestudy]
        f_max = f_maxs['stochastic', casestudy]
        ((df_sp, df_eev, df_ews, df_heur_sp),
         (df_ro, df_mmv, df_mws, df_heur_ro)) =\
            obj_breakdown(casestudy, pftype, f_min, f_max, r_hat)

        df_sp = df_sp.reindex(df_sp.index).fillna(df_sp.min())
        df_eev = df_eev.reindex(df_sp.index).fillna(df_eev.min())
        df_ews = df_ews.reindex(df_sp.index).fillna(df_ews.min())
        
        ax = axes[row, 0]
        df_eev.div(10).plot(color=utcolors[1], ax=ax, label='$z_\\text{EEV}^*$', linewidth=2)
        df_sp.div(10).plot(color=utcolors[3], ax=ax, label='$z_\\text{SP}^*$', linewidth=2, zorder=10)
        df_ews.div(10).plot(color=utcolors[5], ax=ax, label='$z_\\text{EWS}^*$', linewidth=2)
        ax.set_xlabel('$f$')
        ax.set_ylabel(f'Objective Value (GW)\nusing {pftype_names[pftype]} Power Flow')
        ax.set_xlim(xlim['stochastic', casestudy])
        ax.set_ylim(ylim['stochastic', casestudy])
        legend = ax.legend(loc='upper right')
        legend.get_frame().set_edgecolor('k')

        f_min = f_mins['robust', casestudy]
        f_max = f_maxs['robust', casestudy]
        ((df_sp, df_eev, df_ews, df_heur_sp),
         (df_ro, df_mmv, df_mws, df_heur_ro)) =\
            obj_breakdown(casestudy, pftype, f_min, f_max, r_hat)

        df_ro = df_ro.reindex(df_ro.index).fillna(df_ro.min())
        df_mmv = df_mmv.reindex(df_sp.index).fillna(df_mmv.min())
        df_mws = df_mws.reindex(df_sp.index).fillna(df_mws.min())
        
        ax = axes[row, 1]
        df_mmv.div(10).plot(color=utcolors[1], ax=ax, label='$z_\\text{MMV}^*$', linewidth=2)
        df_ro.div(10).plot(color=utcolors[3], ax=ax, label='$z_\\text{RO}^*$', linewidth=2, zorder=10)
        df_mws.div(10).plot(color=utcolors[5], ax=ax, label='$z_\\text{MWS}^*$', linewidth=2)
        ax.set_xlabel('$f$')
        ax.set_ylabel(f'Objective Value (GW)\nusing {pftype_names[pftype]} Power Flow')
        ax.set_xlim(xlim['robust', casestudy])
        ax.set_ylim(ylim['robust', casestudy])
        legend = ax.legend(loc='upper right')
        legend.get_frame().set_edgecolor('k')

    image_jpg = os.path.join(FIG_RESULTS_DIR, f'ijoc-obj-bnds-{casestudy}.jpg')
    image_eps = os.path.join(FIG_RESULTS_DIR, f'ijoc-obj-bnds-{casestudy}.eps')
    plt.savefig(image_jpg, format='jpg', dpi=600)
    plt.savefig(image_eps, format='eps')
    plt.close()
    display(Image(image_jpg))

In [ ]:
for casestudy in casestudies:
    for pftype in pftypes[casestudy]:
        f_min = f_mins['stochastic', casestudy]
        f_max = f_maxs['stochastic', casestudy]
        ((df_sp, df_eev, df_ews, df_heur_sp),
         (df_ro, df_mmv, df_mws, df_heur_ro)) =\
            obj_breakdown(casestudy, pftype, f_min, f_max, r_hat)
        ub_gap = (df_eev - df_sp).div(df_sp).abs()
        lb_gap = (df_ews - df_sp).div(df_sp).abs()
        print('sp', 'ub', casestudy, pftype, ub_gap.idxmax(), round(ub_gap.max(), 4))
        print('sp', 'lb', casestudy, pftype, lb_gap.idxmax(), round(lb_gap.max(), 4))

In [ ]:
for casestudy in casestudies:
    for pftype in pftypes[casestudy]:
        f_min = f_mins['stochastic', casestudy]
        f_max = f_maxs['stochastic', casestudy]
        ((df_sp, df_eev, df_ews, df_heur_sp),
         (df_ro, df_mmv, df_mws, df_heur_ro)) =\
            obj_breakdown(casestudy, pftype, f_min, f_max, r_hat)
        ub_gap = (df_mmv - df_ro).div(df_ro).abs()
        lb_gap = (df_mws - df_ro).div(df_ro).abs()
        print('ro', 'ub', casestudy, pftype, ub_gap.idxmax(), round(ub_gap.max(), 4))
        print('ro', 'lb', casestudy, pftype, lb_gap.idxmax(), round(lb_gap.max(), 4))